In [111]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import torch
import sys
import kaolin as kal
import matplotlib.pyplot as plt
import igl

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'cube'
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


In [112]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  8
Number of faces:  8


In [114]:
# Sample one point
sampled_pts = igl.random_points_on_mesh(1, vertices, faces)
print('Face:', sampled_pts[1])
pt = np.sum(sampled_pts[0][:, np.newaxis] * vertices[faces[sampled_pts[1]]], axis=0)[np.newaxis, :]
print('Point:', pt)
p = mp.plot(vertices, faces, return_plot=True)
p.add_points(pt, shading={"point_size": 0.5, "point_color": "green"})
plt.show()

Face: 2
Point: [[0.9462143  1.         0.38571075]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…

In [124]:
N = 200
sampled_pts = igl.random_points_on_mesh(N, vertices, faces)
pts = np.sum(sampled_pts[0][:,:, np.newaxis] * vertices[faces[sampled_pts[1]]], axis = 1)
p = mp.plot(vertices, faces, return_plot=True)
p.add_points(pts, shading={"point_size": 0.2, "point_color": "green"})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…